<a href="https://colab.research.google.com/github/VictoriaIngham/ITN_Timecourse/blob/main/Final_visualisation_TimeCourse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import Text, VBox, Output
from IPython.display import display
from google.colab import files
import io

# File upload
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv(io.StringIO(uploaded[filename].decode('utf-8')), sep=',')

# Clean column names and strip whitespace
df.columns = df.columns.str.strip()
df['ID'] = df['ID'].astype(str).str.strip()

# Define time points and conditions
timepoints = ['1', '4', '8', '24', '48', '72']
conditions = ['CFPR', 'PYR', 'Combined']
colors = {"CFPR": "#E69F00", "PYR": "#009E73", "Combined": "#CC79A7"}

# Map condition to columns
log2fc_columns = {
    cond: [(tp, f'log2FoldChange.{cond}_vs_Control_{tp}') for tp in timepoints]
    for cond in conditions
}

# Output area for plot + table
out = Output()

def plot_expression(gene_id):
    gene_id = gene_id.strip()
    gene_data = df[df['ID'] == gene_id]

    with out:
        out.clear_output()
        if gene_data.empty:
            print("Gene not found.")
            return

        # Plot expression
        plt.figure(figsize=(10, 6))
        for cond in conditions:
            x = [int(tp) for tp, _ in log2fc_columns[cond]]
            y = gene_data[[col for _, col in log2fc_columns[cond]]].values.flatten()
            sns.lineplot(x=x, y=y, marker='o', label=cond, color=colors[cond])

        plt.axhline(0, linestyle='--', color='gray')
        plt.xlabel('Time (hr)')
        plt.ylabel('log2 Fold Change')
        plt.title(f'Expression of {gene_id} Over Time')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Display log2FC table
        table = {
            f'{cond}_log2FC_{tp}h': gene_data[f'log2FoldChange.{cond}_vs_Control_{tp}'].values[0]
            for cond in conditions for tp in timepoints
        }
        display(pd.DataFrame([table]))

# Text input box for gene ID
text_input = Text(
    value='',
    placeholder='Enter gene ID (e.g. AGAP008212)',
    description='Gene ID:',
    layout={'width': '400px'}
)

def on_text_submit(change):
    plot_expression(change['new'])

text_input.observe(on_text_submit, names='value')  # triggers on typing

# Display input + output
display(VBox([text_input, out]))


Saving final_combined_results_fixed.csv to final_combined_results_fixed (1).csv
